In [1]:
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import scipy
import datetime
import traceback
import pickle
import copy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
RS =  42
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier



This notebook is classifies input text into 4 labels(5 labels I add label UNKNOWN). 
Model is saved by pickle to use it in Flask server. 
Auther Alex Bocharov skype bam271074

In [2]:
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
data_df=pd.read_excel('../content/gdrive/My Drive/dataset_site_generator/eSiteDataset01eng-train.xlsx')
data_df.head()

,Description,Target
0,I want simple business website for a company,business_site
1,I want website of online school,online_school
2,I want landing of my goods,product_landing
3,I want to organize hackhton online,hackathon
4,Bla bla bla bla bla,unknown


In [4]:
val_df=pd.read_excel('../content/gdrive/My Drive/dataset_site_generator/eSiteDataset01eng-val.xlsx')
val_df.head()

,Description,Target
0,I want simple business website for a company,business_site
1,I want website of online school,online_school
2,I want landing of my goods,product_landing
3,I want to organize hackhton online,hackathon
4,Bla bla bla bla bla,unknown


In [5]:
test_df=pd.read_excel('../content/gdrive/My Drive/dataset_site_generator/eSiteDataset01eng-test.xlsx')
test_df.head()

,Description,Target
0,site online prices about servises products,NaN
1,I wanna sale my product online,NaN
2,study online learn courses,NaN
3,online competition of young programmers and de...,NaN
4,stady site for company,NaN


In [7]:
 !pip3 list 
 #check versions

Package                  Version        
------------------------ ---------------
absl-py                  0.9.0          
alabaster                0.7.12         
albumentations           0.1.12         
altair                   4.1.0          
asgiref                  3.2.10         
astor                    0.8.1          
astropy                  4.0.1.post1    
astunparse               1.6.3          
atari-py                 0.2.6          
atomicwrites             1.4.0          
attrs                    19.3.0         
audioread                2.1.8          
autograd                 1.3            
Babel                    2.8.0          
backcall                 0.2.0          
beautifulsoup4           4.6.3          
bleach                   3.1.5          
blis                     0.4.1          
bokeh                    2.1.1          
boto                     2.49.0         
boto3                    1.14.30        
botocore                 1.17.30        
Bottleneck      

In [6]:
TOKEN_RE = re.compile(r'[\w\d]+')  #regular expression to start with
#TOKEN_RE = re.compile(r'[a-z]+|-?\d*[.,]?\d+|\S')

In [7]:
def tokenize_text_simple_regex(txt, min_token_size=4):
    """ This func tokenize text with TOKEN_RE applied ealier """
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]


In [8]:

#divide dataset to train and test
df_x, x_val, df_y, y_val=train_test_split(data_df['Description'],\
                                                data_df['Target'], test_size=0.2,random_state=RS)

In [9]:
pl_clf = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
                                                      max_df=0.96,
                                                      min_df=4)),
                             ('cls', RandomForestClassifier(n_estimators=150,max_depth=250,
                                                            n_jobs=-1, \
                                                  verbose=True,random_state=RS))),
                  verbose=True)

In [10]:
pl_clf.fit(df_x,df_y);

[Pipeline] .............. (step 1 of 2) Processing vect, total=   0.0s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s


[Pipeline] ............... (step 2 of 2) Processing cls, total=   0.4s


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.3s finished


In [11]:
print('Скор на трейне',pl_clf.score(df_x, df_y))
print('Скор по валидации',pl_clf.score(x_val, y_val))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 150 out of 150 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 150 out of 150 | elapsed:    0.0s finished


Скор на трейне 0.9367088607594937
Скор по валидации 0.7


In [12]:
from sklearn.metrics import accuracy_score

print('Доля верных ответов на трейне', accuracy_score(data_df['Target'], pl_clf.predict(data_df['Description'])))
print()
print('Доля верных ответов на валидации', accuracy_score(val_df['Target'], pl_clf.predict(val_df['Description'])))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 150 out of 150 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 150 out of 150 | elapsed:    0.0s finished


Доля верных ответов на трейне 0.8888888888888888

Доля верных ответов на валидации 0.8793103448275862


In [13]:
#let s save our model with pickle
filename = 'model_site_generator01.pickle'
pickle.dump(pl_clf, open(filename, 'wb'))

# This is example of code how to load saved model
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# prediction = loaded_model.predict(X_test)
# print(prediction)

In [14]:
!ls

gdrive	model_site_generator01.pickle  sample_data


In [15]:
from google.colab import files

files.download('model_site_generator01.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
#let s save tokenizer with pickle
filename = 'tokenize_text_simple_regex01.pickle'
pickle.dump(tokenize_text_simple_regex, open(filename, 'wb'))

In [17]:
!ls

gdrive			       sample_data
model_site_generator01.pickle  tokenize_text_simple_regex01.pickle


In [18]:
from google.colab import files

files.download('tokenize_text_simple_regex01.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>